In [210]:
import pandas as pd
import pytz

In [211]:
tempo = pd.read_csv('../Bases/Raw/INMET_SE_SP_A701_SAO PAULO - MIRANTE_01-01-2019_A_31-12-2019.CSV', sep=';', header=8, encoding='utf-8')

In [212]:
tempo = tempo[['Data','Hora UTC','PRECIPITACAO TOTAL, HORARIO (mm)','TEMPERATURA MAXIMA NA HORA ANT. (AUT) (°C)','TEMPERATURA MINIMA NA HORA ANT. (AUT) (°C)','UMIDADE RELATIVA DO AR, HORARIA (%)','VENTO, VELOCIDADE HORARIA (m/s)']]
tempo = tempo.loc[(tempo['Data'] >= '2019/06/01') & (tempo['Data'] <= '2019/10/10')]

In [213]:

tempo['Datetime'] = pd.to_datetime(tempo['Data']+' '+tempo['Hora UTC'])

In [214]:
tempo['Datetime'] = tempo['Datetime'].dt.tz_convert(pytz.timezone('America/Sao_Paulo'))
tempo['Datetime'] = tempo['Datetime'].dt.tz_localize(None)

In [215]:
tempo["Temperatura Media"] = (tempo["TEMPERATURA MAXIMA NA HORA ANT. (AUT) (°C)"].str.replace(",", ".").astype(float) + tempo["TEMPERATURA MINIMA NA HORA ANT. (AUT) (°C)"].str.replace(",", ".").astype(float))/2

In [216]:
tempo.drop(["Data", "Hora UTC", "TEMPERATURA MAXIMA NA HORA ANT. (AUT) (°C)", "TEMPERATURA MINIMA NA HORA ANT. (AUT) (°C)"], axis=1, inplace=True)

In [217]:
tempo.columns = ["Precipitacao", "Umidade", "Vento", "Datetime", "Temperatura"]

In [218]:
tempo["Vento"] = tempo["Vento"].str.replace(",", ".").astype(float)
tempo["Precipitacao"] = tempo["Precipitacao"].str.replace(",", ".").astype(float)

In [219]:
tempo.dtypes

Precipitacao           float64
Umidade                  int64
Vento                  float64
Datetime        datetime64[ns]
Temperatura            float64
dtype: object

In [220]:
tempo

,Precipitacao,Umidade,Vento,Datetime,Temperatura
3624,0.0,51,2.5,2019-05-31 21:00:00,24.20
3625,3.8,85,4.5,2019-05-31 22:00:00,21.30
3626,3.0,89,0.3,2019-05-31 23:00:00,18.45
3627,0.0,88,0.0,2019-06-01 00:00:00,18.70
3628,0.0,86,0.4,2019-06-01 01:00:00,18.80
...,...,...,...,...,...
6787,0.0,59,3.8,2019-10-10 16:00:00,26.95
6788,0.0,70,3.4,2019-10-10 17:00:00,23.55
6789,0.0,77,1.8,2019-10-10 18:00:00,21.25
6790,0.0,81,2.0,2019-10-10 19:00:00,20.00


In [221]:
tempo.to_json("../Bases/Cleaned/weather.json")

In [222]:
df33642 = pd.read_csv("../Bases/Cleaned/rota33642.csv")
df32893 = pd.read_csv("../Bases/Cleaned/rota32893.csv")

In [223]:
df33642["data_partida2"] = pd.to_datetime(df33642["data_partida"])
df33642["data_partida2"] = df33642["data_partida2"].dt.round("H")

df32893["data_partida2"] = pd.to_datetime(df32893["data_partida"])
df32893["data_partida2"] = df32893["data_partida2"].dt.round("H")

In [224]:
df33642_merged = df33642.merge(tempo, left_on="data_partida2", right_on="Datetime", how="left")
df32893_merged = df32893.merge(tempo, left_on="data_partida2", right_on="Datetime", how="left")

In [225]:
df33642_merged.drop(['data_partida2','Datetime'],axis=1,inplace=True)
df32893_merged.drop(['data_partida2','Datetime'],axis=1,inplace=True)

In [226]:
df33642_merged.to_csv("../Bases/Cleaned/rota33642_weather.csv",index=False)
df32893_merged.to_csv("../Bases/Cleaned/rota32893_weather.csv",index=False)